In [1]:
#Import all the needed libraries
import os
import numpy as np
import pandas as pd
import warnings

warnings.filterwarnings("ignore")

In [2]:
#Check the current working directory
os.getcwd()

'C:\\Users\\Имя'

In [3]:
# Define all the helping functions
####################################################
# Import packages
####################################################
import numpy as np
import copy
import pandas as pd
from plotnine import *


####################################################
# Define global vars
####################################################
color = ["#3a5e8cFF", "#10a53dFF", "#541352FF", "#ffcf20FF", "#2f9aa0FF"]


####################################################
# Define helper functions
####################################################
def seq(start, stop, by, round_n=3):
    return [round(x, round_n) for x in list(np.arange(start, stop, by))]


def skew(l, round_n=3):
    return round((np.mean(l) - np.median(l)) / np.std(l), round_n)


def lspline(series, knots):
    def knot_ceil(vector, knot):
        vector_copy = copy.deepcopy(vector)
        vector_copy[vector_copy > knot] = knot
        return vector_copy

    if type(knots) != list:
        knots = [knots]
    design_matrix = None
    vector = series.values

    for i in range(len(knots)):
        # print(i)
        # print(vector)
        if i == 0:
            column = knot_ceil(vector, knots[i])
        else:
            column = knot_ceil(vector, knots[i] - knots[i - 1])
        # print(column)
        if i == 0:
            design_matrix = column
        else:
            design_matrix = np.column_stack((design_matrix, column))
        # print(design_matrix)
        vector = vector - column
    design_matrix = np.column_stack((design_matrix, vector))
    # print(design_matrix)
    return design_matrix

def create_calibration_plot(
    data,
    file_name,
    prob_var,
    actual_var,
    y_lab="Actual event probability",
    n_bins=10,
    breaks=None,
):
    if breaks is None:
        breaks = np.around(
            np.linspace(0, (n_bins + 1) / 10, num=n_bins + 1, endpoint=False),
            decimals=1,
        ).tolist()

    data["prob_bin"] = pd.cut(data[prob_var], breaks, right=True, include_lowest=True)

    binned_data = (
        data.groupby("prob_bin")
        .agg(
            mean_prob=(prob_var, "mean"),
            mean_actual=(actual_var, "mean"),
            n=(actual_var, "size"),
        )
        .reset_index()
    )

    plot = (
        ggplot(binned_data, aes("mean_prob", "mean_actual"))
        + geom_line(color=color[0], size=1, show_legend=True)
        + geom_point(
            color=color[0],
            size=1,
            alpha=0.7,
            show_legend=False,
            na_rm=True,
        )
        + geom_segment(
            x=min(breaks),
            xend=max(breaks),
            y=min(breaks),
            yend=max(breaks),
            color=color[1],
            size=0.5,
        )
        + theme_bw()
        + labs(x="Predicted event probability", y=y_lab)
        + coord_cartesian(xlim=(0, 1), ylim=(0, 1))
        + expand_limits(x=0.01, y=0.01)
        + scale_y_continuous(expand=(0.01, 0.01), breaks=(seq(0, 1.1, 0.1)))
        + scale_x_continuous(expand=(0.01, 0.01), breaks=(seq(0, 1.1, 0.1)))
    )

    return plot


def poly(x, degree=1):
    """
    Fit Polynomial
    These are non orthogonal factors, but it may not matter if
    we only need this for predictions (without interpreting the
    coefficients) or visualisation.
    """
    d = {}
    for i in range(degree+1):
        if i == 1:
            d['x'] = x
        else:
            d[f'x**{i}'] = np.power(x, i)
    return pd.DataFrame(d)

# Part I - Data cleaning and loading

For my project I use data set for Airbnb listings for San Diego available from Inside Airbnb website. Initial dataset has 9673 observations and is collected for the data 25th of September 2021.

In [4]:
#First I just import the raw data
data = pd.read_csv('listings.csv', delimiter = ',',dtype ='unicode')

In [5]:
#Checking what I imorted
data.head()

,id,listing_url,scrape_id,last_scraped,name,description,neighborhood_overview,picture_url,host_id,host_url,...,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,5570,https://www.airbnb.com/rooms/5570,20210925150545,2021-09-25,Ocean front condo on the sand,Missionbeachretreat. Great Ocean Front Condo...,While some may compare San Diego’s Mission Bea...,https://a0.muscache.com/pictures/d3342744-140a...,8435,https://www.airbnb.com/users/show/8435,...,NaN,NaN,NaN,NaN,f,3,3,0,0,NaN
1,29967,https://www.airbnb.com/rooms/29967,20210925150545,2021-09-26,"Great home, 10 min walk to Beach","This 2 bedroom, 2 full bathroom home offers al...",NaN,https://a0.muscache.com/pictures/76103867/c9b5...,129123,https://www.airbnb.com/users/show/129123,...,4.93,4.83,4.83,NaN,f,5,5,0,0,0.52
2,38245,https://www.airbnb.com/rooms/38245,20210925150545,2021-09-26,Point Loma: Den downstairs,Stay in a 1928 Spanish Style home. Den is dow...,"This neighborhood is an expensive, high-end Po...",https://a0.muscache.com/pictures/f3073822-e493...,164137,https://www.airbnb.com/users/show/164137,...,4.79,4.61,4.4,NaN,f,3,0,3,0,1.46
3,39516,https://www.airbnb.com/rooms/39516,20210925150545,2021-09-26,Art Studio Retreat/Rustic Cabin,We love having explorers from all over the wor...,"We love our eclectic neighborhood, with a smal...",https://a0.muscache.com/pictures/2ee5c482-c48f...,169649,https://www.airbnb.com/users/show/169649,...,4.96,4.9,4.94,NaN,f,1,1,0,0,3.38
4,54001,https://www.airbnb.com/rooms/54001,20210925150545,2021-09-26,"La Jolla Cottage Blocks to Ocn; 2Bdms, 1Bth.","Separate charming cottage behind main home, 2 ...","Tranquility, stately trees, meditative, zen-fu...",https://a0.muscache.com/pictures/47035b04-633c...,252692,https://www.airbnb.com/users/show/252692,...,4.97,4.97,4.88,NaN,f,2,1,1,0,5.47


# Getting rid from not needed columns

In [6]:
#Look at all columns names
for col in data:
    print(col)

id
listing_url
scrape_id
last_scraped
name
description
neighborhood_overview
picture_url
host_id
host_url
host_name
host_since
host_location
host_about
host_response_time
host_response_rate
host_acceptance_rate
host_is_superhost
host_thumbnail_url
host_picture_url
host_neighbourhood
host_listings_count
host_total_listings_count
host_verifications
host_has_profile_pic
host_identity_verified
neighbourhood
neighbourhood_cleansed
neighbourhood_group_cleansed
latitude
longitude
property_type
room_type
accommodates
bathrooms
bathrooms_text
bedrooms
beds
amenities
price
minimum_nights
maximum_nights
minimum_minimum_nights
maximum_minimum_nights
minimum_maximum_nights
maximum_maximum_nights
minimum_nights_avg_ntm
maximum_nights_avg_ntm
calendar_updated
has_availability
availability_30
availability_60
availability_90
availability_365
calendar_last_scraped
number_of_reviews
number_of_reviews_ltm
number_of_reviews_l30d
first_review
last_review
review_scores_rating
review_scores_accuracy
review_sc

In [7]:
# First define columns which aren't needed just logically
drops = [
    'listing_url',
    'name', #no information I could use witout text analysis
    'host_location', #I already know I look at San Diego
    'scrape_id',
    'last_scraped',
    'description',
    'neighborhood_overview',
    'picture_url',
    'host_url',
    'host_name',
    'host_about',
    'host_thumbnail_url',
    'host_picture_url',
    'neighbourhood_group_cleansed',
    'bathrooms', #empty
    'minimum_minimum_nights',
    'maximum_minimum_nights', 
    'minimum_maximum_nights',
    'maximum_maximum_nights',
    'minimum_nights_avg_ntm',
    'maximum_nights_avg_ntm',
    'calendar_updated',
    'calendar_last_scraped', 
    'license'   
]
data.drop(columns = drops, inplace = True) #drop all not needed columns

In [8]:
#look how it looks now
data.head()

,id,host_id,host_since,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_neighbourhood,host_listings_count,host_total_listings_count,...,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,5570,8435,2009-02-23,NaN,NaN,100%,f,Mission Beach,3,3,...,NaN,NaN,NaN,NaN,f,3,3,0,0,NaN
1,29967,129123,2010-05-21,within a few hours,100%,63%,f,Pacific Beach,6,6,...,4.95,4.93,4.83,4.83,f,5,5,0,0,0.52
2,38245,164137,2010-07-12,within an hour,100%,50%,f,Loma Portal,3,3,...,4.71,4.79,4.61,4.4,f,3,0,3,0,1.46
3,39516,169649,2010-07-17,NaN,NaN,NaN,t,North Park,1,1,...,4.97,4.96,4.9,4.94,f,1,1,0,0,3.38
4,54001,252692,2010-10-04,within a few hours,100%,84%,t,La Jolla,5,5,...,4.96,4.97,4.97,4.88,f,2,1,1,0,5.47


In [9]:
[col for col in data.columns if data[col].isnull().any()] # Look at the columns which have NAs

['host_since',
 'host_response_time',
 'host_response_rate',
 'host_acceptance_rate',
 'host_is_superhost',
 'host_neighbourhood',
 'host_listings_count',
 'host_total_listings_count',
 'host_has_profile_pic',
 'host_identity_verified',
 'neighbourhood',
 'bathrooms_text',
 'bedrooms',
 'beds',
 'first_review',
 'last_review',
 'review_scores_rating',
 'review_scores_accuracy',
 'review_scores_cleanliness',
 'review_scores_checkin',
 'review_scores_communication',
 'review_scores_location',
 'review_scores_value',
 'reviews_per_month']

In [10]:
#Function to define table with missing values
def missing_zero_values_table(df):
        zero_val = (df == 0.00).astype(int).sum(axis=0)
        mis_val = df.isnull().sum()
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        mz_table = pd.concat([zero_val, mis_val, mis_val_percent], axis=1)
        mz_table = mz_table.rename(
        columns = {0 : 'Zero Values', 1 : 'Missing Values', 2 : '% of Total Values'})
        mz_table['Total Zero Missing Values'] = mz_table['Zero Values'] + mz_table['Missing Values']
        mz_table['% Total Zero Missing Values'] = 100 * mz_table['Total Zero Missing Values'] / len(df)
        mz_table['Data Type'] = df.dtypes
        mz_table = mz_table[
            mz_table.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns and " + str(df.shape[0]) + " Rows.\n"      
            "There are " + str(mz_table.shape[0]) +
              " columns that have missing values.")
#         mz_table.to_excel('D:/sampledata/missing_and_zero_values.xlsx', freeze_panes=(1,0), index = False)
        return mz_table

In [11]:
missing_zero_values_table(data) #print table with missing values

Your selected dataframe has 50 columns and 9672 Rows.
There are 24 columns that have missing values.


,Zero Values,Missing Values,% of Total Values,Total Zero Missing Values,% Total Zero Missing Values,Data Type
neighbourhood,0,2787,28.8,2787,28.8,object
review_scores_value,0,1249,12.9,1249,12.9,object
review_scores_location,0,1249,12.9,1249,12.9,object
review_scores_checkin,0,1249,12.9,1249,12.9,object
review_scores_communication,0,1247,12.9,1247,12.9,object
review_scores_cleanliness,0,1247,12.9,1247,12.9,object
review_scores_accuracy,0,1247,12.9,1247,12.9,object
reviews_per_month,0,1218,12.6,1218,12.6,object
first_review,0,1218,12.6,1218,12.6,object
review_scores_rating,0,1218,12.6,1218,12.6,object


I can see that neigbourhood has a lot of missing data (almost 1/3 of the observations). Looking more carefully into the data I can see that neighourhood information is much better depicted in neighourborhood cleansed (and it has no missing values). Thus, I drop variable neigbourhood. Host neighorborhood sometimes also has missing values. In general it looks very similaer to neighbourhood cleansed, but with more missing values for reason I couldn't identify. I decided to only keep neighborhood cleansed.


In [12]:
data.drop(columns = ['neighbourhood', 'host_neighbourhood'], inplace=True )

In [13]:
data.head()

,id,host_id,host_since,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_listings_count,host_total_listings_count,host_verifications,...,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,5570,8435,2009-02-23,NaN,NaN,100%,f,3,3,"['email', 'phone', 'kba']",...,NaN,NaN,NaN,NaN,f,3,3,0,0,NaN
1,29967,129123,2010-05-21,within a few hours,100%,63%,f,6,6,"['email', 'phone', 'reviews', 'kba', 'work_ema...",...,4.95,4.93,4.83,4.83,f,5,5,0,0,0.52
2,38245,164137,2010-07-12,within an hour,100%,50%,f,3,3,"['email', 'phone', 'reviews', 'kba']",...,4.71,4.79,4.61,4.4,f,3,0,3,0,1.46
3,39516,169649,2010-07-17,NaN,NaN,NaN,t,1,1,"['email', 'phone', 'reviews', 'jumio', 'offlin...",...,4.97,4.96,4.9,4.94,f,1,1,0,0,3.38
4,54001,252692,2010-10-04,within a few hours,100%,84%,t,5,5,"['email', 'phone', 'reviews', 'kba']",...,4.96,4.97,4.97,4.88,f,2,1,1,0,5.47


In [14]:
#Save this data after first cleaning and don't touch it
data.to_csv('airbnb_sd_listings.csv', index='False')

In [15]:
#Open data set
df = pd.read_csv('airbnb_sd_listings.csv', delimiter = ',')

In [16]:
df.head() #check how it looks

,Unnamed: 0,id,host_id,host_since,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_listings_count,host_total_listings_count,...,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,0,5570,8435,2009-02-23,NaN,NaN,100%,f,3.0,3.0,...,NaN,NaN,NaN,NaN,f,3,3,0,0,NaN
1,1,29967,129123,2010-05-21,within a few hours,100%,63%,f,6.0,6.0,...,4.95,4.93,4.83,4.83,f,5,5,0,0,0.52
2,2,38245,164137,2010-07-12,within an hour,100%,50%,f,3.0,3.0,...,4.71,4.79,4.61,4.40,f,3,0,3,0,1.46
3,3,39516,169649,2010-07-17,NaN,NaN,NaN,t,1.0,1.0,...,4.97,4.96,4.90,4.94,f,1,1,0,0,3.38
4,4,54001,252692,2010-10-04,within a few hours,100%,84%,t,5.0,5.0,...,4.96,4.97,4.97,4.88,f,2,1,1,0,5.47


# Check duplicate values

In [17]:
df.shape #store df as a number of rows and columns

(9672, 49)

In [18]:
df = df.drop_duplicates()

In [19]:
df.shape #check rows and columns again - great there was no duplicates

(9672, 49)

# Separate data types

In [20]:
#First, drop id-s which are not characteristic of number - "broken lines"
df.id = pd.to_numeric(df.id, errors = 'coerce') #coerce means that it will try to find numbers which are not definied as numbers and replace this with number


In [21]:
df.shape #nothing was a problem

(9672, 49)

In [22]:
df = df[df.id.notna()] #leave only those without missing values

In [23]:
df.shape

(9672, 49)

In [24]:
# I checked the fact of missing values but that I knew from before that for id there is no missing values

Now I will check data types. There can be the following types I should think about



Numerical (Continuous/Discrete)


Categorical (Binary/Ordinal/Nominal)


Mixed Type


Time

In [25]:
# First categorical variables
cat_df = df.select_dtypes(include=['object', 'bool'])
cat_cols = df.select_dtypes(include=['object', 'bool']).columns.values

In [26]:
cat_cols # look which one are categorical

array(['host_since', 'host_response_time', 'host_response_rate',
       'host_acceptance_rate', 'host_is_superhost', 'host_verifications',
       'host_has_profile_pic', 'host_identity_verified',
       'neighbourhood_cleansed', 'property_type', 'room_type',
       'bathrooms_text', 'amenities', 'price', 'has_availability',
       'first_review', 'last_review', 'instant_bookable'], dtype=object)

In [27]:
df.shape

(9672, 49)

In [28]:
#Now numerical variables
num_df = df.select_dtypes(include=['int', 'int64','float'])
num_cols = df.select_dtypes(include=['int', 'int64','float']).columns.values

In [29]:
num_cols #which once are numerical

array(['Unnamed: 0', 'id', 'host_id', 'host_listings_count',
       'host_total_listings_count', 'latitude', 'longitude',
       'accommodates', 'bedrooms', 'beds', 'minimum_nights',
       'maximum_nights', 'availability_30', 'availability_60',
       'availability_90', 'availability_365', 'number_of_reviews',
       'number_of_reviews_ltm', 'number_of_reviews_l30d',
       'review_scores_rating', 'review_scores_accuracy',
       'review_scores_cleanliness', 'review_scores_checkin',
       'review_scores_communication', 'review_scores_location',
       'review_scores_value', 'calculated_host_listings_count',
       'calculated_host_listings_count_entire_homes',
       'calculated_host_listings_count_private_rooms',
       'calculated_host_listings_count_shared_rooms', 'reviews_per_month'],
      dtype=object)

In [30]:
# Not forget to check that all features are accounted for
len(num_cols) + len(cat_cols) == df.shape[1] # After debugging and adding int64 everything works

True

In [31]:
#Let's look at numerical variables
num_df.head().transpose() # Looks okay, there are missing values but with them I will deal later

,0,1,2,3,4
Unnamed: 0,0.00000,1.00000,2.00000,3.00000,4.00000
id,5570.00000,29967.00000,38245.00000,39516.00000,54001.00000
host_id,8435.00000,129123.00000,164137.00000,169649.00000,252692.00000
host_listings_count,3.00000,6.00000,3.00000,1.00000,5.00000
host_total_listings_count,3.00000,6.00000,3.00000,1.00000,5.00000
latitude,32.78481,32.80751,32.74217,32.73004,32.81301
longitude,-117.25313,-117.25760,-117.21931,-117.11932,-117.26856
accommodates,6.00000,8.00000,1.00000,2.00000,4.00000
bedrooms,2.00000,2.00000,1.00000,1.00000,2.00000
beds,4.00000,3.00000,1.00000,1.00000,3.00000


In [32]:
# Let's look at categorical variables
cat_df.head().transpose()

,0,1,2,3,4
host_since,2009-02-23,2010-05-21,2010-07-12,2010-07-17,2010-10-04
host_response_time,NaN,within a few hours,within an hour,NaN,within a few hours
host_response_rate,NaN,100%,100%,NaN,100%
host_acceptance_rate,100%,63%,50%,NaN,84%
host_is_superhost,f,f,f,t,t
host_verifications,"['email', 'phone', 'kba']","['email', 'phone', 'reviews', 'kba', 'work_ema...","['email', 'phone', 'reviews', 'kba']","['email', 'phone', 'reviews', 'jumio', 'offlin...","['email', 'phone', 'reviews', 'kba']"
host_has_profile_pic,t,t,t,t,t
host_identity_verified,t,t,t,t,t
neighbourhood_cleansed,Mission Bay,Pacific Beach,Roseville,North Hills,La Jolla
property_type,Entire condominium (condo),Entire bungalow,Private room in residential home,Tiny house,Entire residential home


Categorical variables don't look that good. There are the following problems:


1) There are time variables

2) I want to create dummies for binary categorical variables

3) I may want to calculate number of amenities

4) I want to separate price in numerical variable deleting the dollar sign

5) I want to make percentages a numerical variables in percentage points

6) I want to delete word bath in bathrooms to get the number of bathrooms 

7) I want to do smth with variables containing multiple variables

# Tranforming and cleaning categorical variables

Time variables

In [33]:
# First, I want to create a reference for the 'time since'. For this, I create variable today
# Today refers to 25th of September 2021 for which the data from San Diego is scraped
today = pd.to_datetime('2021-09-25')

In [34]:
time_var = ['host_since','first_review', 'last_review'] #putting together all time variables

In [35]:
for col in time_var: #transfering this columns to date time in data frame
    df[col] = pd.to_datetime(df[col], infer_datetime_format=True)

In [36]:
df.host_since.describe()

count                    9581
unique                   2489
top       2016-08-05 00:00:00
freq                      202
first     2009-02-23 00:00:00
last      2021-09-22 00:00:00
Name: host_since, dtype: object

In [37]:
# Creating date features for each of the time var columns
for col in time_var:
    df[col+"_days"] = (today - df[col]).apply(lambda x: x.days)

In [38]:
df.head() #look what I got - looks like what is needed

,Unnamed: 0,id,host_id,host_since,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_listings_count,host_total_listings_count,...,review_scores_value,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month,host_since_days,first_review_days,last_review_days
0,0,5570,8435,2009-02-23,NaN,NaN,100%,f,3.0,3.0,...,NaN,f,3,3,0,0,NaN,4597.0,NaN,NaN
1,1,29967,129123,2010-05-21,within a few hours,100%,63%,f,6.0,6.0,...,4.83,f,5,5,0,0,0.52,4145.0,4076.0,47.0
2,2,38245,164137,2010-07-12,within an hour,100%,50%,f,3.0,3.0,...,4.40,f,3,0,3,0,1.46,4093.0,3002.0,41.0
3,3,39516,169649,2010-07-17,NaN,NaN,NaN,t,1.0,1.0,...,4.94,f,1,1,0,0,3.38,4088.0,3549.0,558.0
4,4,54001,252692,2010-10-04,within a few hours,100%,84%,t,5.0,5.0,...,4.88,f,2,1,1,0,5.47,4009.0,1489.0,5.0


In [39]:
df.head()

,Unnamed: 0,id,host_id,host_since,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_listings_count,host_total_listings_count,...,review_scores_value,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month,host_since_days,first_review_days,last_review_days
0,0,5570,8435,2009-02-23,NaN,NaN,100%,f,3.0,3.0,...,NaN,f,3,3,0,0,NaN,4597.0,NaN,NaN
1,1,29967,129123,2010-05-21,within a few hours,100%,63%,f,6.0,6.0,...,4.83,f,5,5,0,0,0.52,4145.0,4076.0,47.0
2,2,38245,164137,2010-07-12,within an hour,100%,50%,f,3.0,3.0,...,4.40,f,3,0,3,0,1.46,4093.0,3002.0,41.0
3,3,39516,169649,2010-07-17,NaN,NaN,NaN,t,1.0,1.0,...,4.94,f,1,1,0,0,3.38,4088.0,3549.0,558.0
4,4,54001,252692,2010-10-04,within a few hours,100%,84%,t,5.0,5.0,...,4.88,f,2,1,1,0,5.47,4009.0,1489.0,5.0


In [40]:
# Now time var colums can be dropped from initial data set
df.drop(columns = time_var, inplace=True)

In [41]:
df.head() #check number of columns now - good

,Unnamed: 0,id,host_id,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_listings_count,host_total_listings_count,host_verifications,...,review_scores_value,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month,host_since_days,first_review_days,last_review_days
0,0,5570,8435,NaN,NaN,100%,f,3.0,3.0,"['email', 'phone', 'kba']",...,NaN,f,3,3,0,0,NaN,4597.0,NaN,NaN
1,1,29967,129123,within a few hours,100%,63%,f,6.0,6.0,"['email', 'phone', 'reviews', 'kba', 'work_ema...",...,4.83,f,5,5,0,0,0.52,4145.0,4076.0,47.0
2,2,38245,164137,within an hour,100%,50%,f,3.0,3.0,"['email', 'phone', 'reviews', 'kba']",...,4.40,f,3,0,3,0,1.46,4093.0,3002.0,41.0
3,3,39516,169649,NaN,NaN,NaN,t,1.0,1.0,"['email', 'phone', 'reviews', 'jumio', 'offlin...",...,4.94,f,1,1,0,0,3.38,4088.0,3549.0,558.0
4,4,54001,252692,within a few hours,100%,84%,t,5.0,5.0,"['email', 'phone', 'reviews', 'kba']",...,4.88,f,2,1,1,0,5.47,4009.0,1489.0,5.0


Creating dummy variables for binary categorical variables

In [42]:
# I have 5 columns with f,t for which I want to have 0, 1
bi_col = ['host_is_superhost','host_has_profile_pic','host_identity_verified','has_availability','instant_bookable']


In [43]:
# Now transform df in needed way
for col in bi_col:
    df[col] = df[col].map({'f':0, 't':1})

In [44]:
# let's see
df.instant_bookable.head(20) #looks good

0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
11    0
12    1
13    1
14    1
15    0
16    0
17    0
18    0
19    1
Name: instant_bookable, dtype: int64

Price variable transformation

Here the problem was that numbers with ',' (for example, 2020,564) weren't recongised as numeric and were transfered as NA 

In [45]:
df.price = df.price.str.replace("\\$", "").str.replace(',', '') #replace dollar sign and account for the fact that numbers can 
    #be higher than in London data and have ","

In [46]:
df.price.head(100)

0     2050.00
1      225.00
2       74.00
3       74.00
4      258.00
       ...   
95     700.00
96      61.00
97      62.00
98     128.00
99    1324.00
Name: price, Length: 100, dtype: object

Making percentage a numerical variable

In [47]:
df.host_response_rate.head()

0     NaN
1    100%
2    100%
3     NaN
4    100%
Name: host_response_rate, dtype: object

In [48]:
def repl(m): return m.group(1)


df['host_response_rate'] = df['host_response_rate'].str.replace(
   r'(\d+)%', repl).astype(float)

In [49]:
df.host_response_rate.head() #done

0      NaN
1    100.0
2    100.0
3      NaN
4    100.0
Name: host_response_rate, dtype: float64

Getting number of bathes

In [50]:
#Firstly, in the data there is one strange observation with 27.5 baths for apartment with 2 bedrooms - drop it
df.shape #check how df looks


(9672, 49)

In [51]:
df = df[df.bathrooms_text != '27.5 baths'] # get df without these 27.5 baths

In [52]:
df.shape #great I got what I wanted

(9671, 49)

In [59]:
# I have depicted some baths decoded as as Half bath, private half bath and shared half bath
# I want to replace it
df['bathrooms_text'] = np.where(df['bathrooms_text'] == 'Half-bath', 0.5, df['bathrooms_text'])
df['bathrooms_text'] = np.where(df['bathrooms_text'] == 'Shared half-bath', 0.5, df['bathrooms_text'])
df['bathrooms_text'] = np.where(df['bathrooms_text'] == 'Private half-bath', 0.5, df['bathrooms_text'])

In [60]:
df['bathrooms_text'].describe()

count       9666
unique        36
top       1 bath
freq        4465
Name: bathrooms_text, dtype: object

In [55]:
#df.bath = df.bathrooms_text.astype(str).str[0] #get first value from column - that will be number of bathes
#okay not that easy

In [56]:
#df.bath.head(100)